# Scooter Exploration

> In this chapter Scooter data is explored 
- prettify: true

<details open>
<summary>

## Introduction

</summary>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/karpatic/karpatic/main?filepath=src%2Fipynb%2Fdatalabs%2F01_scooter_exploration.ipynb)
[![Binder](https://pete88b.github.io/fastpages/assets/badges/colab.svg)](https://colab.research.google.com/github/karpatic/karpatic/blob/main/src/ipynb/datalabs/01_scooter_exploration.ipynb)
[![Binder](https://pete88b.github.io/fastpages/assets/badges/github.svg)](https://github.com/karpatic/karpatic/blob/main/src/ipynb/datalabs/01_scooter_exploration.ipynb)
[![Open Source Love svg3](https://badges.frapsoft.com/os/v3/open-source.svg?v=103)](https://github.com/ellerbrock/open-source-badges/)

[![NPM License](https://img.shields.io/npm/l/all-contributors.svg?style=flat)](https://github.com/karpatic/karpatic/blob/main/LICENSE)
[![Active](http://img.shields.io/badge/Status-Active-green.svg)](https://karpatic.github.io) 
[![GitHub last commit](https://img.shields.io/github/last-commit/karpatic/karpatic.svg?style=flat)]()  

[![GitHub stars](https://img.shields.io/github/stars/karpatic/karpatic.svg?style=social&label=Star)](https://github.com/karpatic/karpatic) 
[![GitHub watchers](https://img.shields.io/github/watchers/karpatic/karpatic.svg?style=social&label=Watch)](https://github.com/karpatic/karpatic) 
[![GitHub forks](https://img.shields.io/github/forks/karpatic/karpatic.svg?style=social&label=Fork)](https://github.com/karpatic/karpatic) 
[![GitHub followers](https://img.shields.io/github/followers/karpatic.svg?style=social&label=Follow)](https://github.com/karpatic/karpatic)  

(((info::This notebook was made in part by interns Brian Kelly, Michael Vandi. Read their final article [An Analysis of Baltimore City E-Scooter Distribution](https://bniajfi.org/2021/02/08/social-media-alert-system-for-covid-19-illicit-behavior/) )))


**Dataset:**
Scooter data: 
- Routes: 3 months (September to August 2019)
- Deployment/
- Routes
- Trip origins-destinations by month

In [ ]:
# do you see this? - Carlos

In [ ]:
# Hello world! - Michael

In [ ]:
# Checking - Brian

</details>
<details>
<summary>

## Local File Access (Optional)

</summary>

In [ ]:
# hide_output
# (Optional) Run this cell to gain access to Google Drive (Colabs only) 
from google.colab import drive

# Colabs operates in a virtualized enviornment
# Colabs default directory is at ~/content.
# We mount Drive into a temporary folder at '~/content/drive' 

drive.mount('/content/drive')

File path's will vary

In [ ]:
# cd ./drive/'My Drive'/BNIA/responsive_records/Routes

In [ ]:
# cd ../content/drive/My Drive/DATA/scooter

In [ ]:
# Michael's Directory
# cd ./drive/'My Drive'/BNIA/'Scooter Use Data'/BNIA

In [ ]:
cd ./Routes

In [ ]:
# hide_output
! ls

In [ ]:
# hide_output
!cd ../ && ls

</details>
<details>
<summary>

## Installs

</summary>

In [1]:
#show_input 
!pip install dexplot
!pip install folium
!pip install geopandas
!pip install ipyleaflet
!pip install gpdvega
!pip install dataplay

</details>
<details>
<summary>

## **Imports**

</summary>

In [ ]:
#show_input 
import os
import pandas as pd
import geopandas as gpd
import dexplot as dxp
import folium as fol
import json
import altair as alt
import gpdvega
import dataplay
# These imports will handle everything
import os
import sys
import csv
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import psycopg2
import pyproj
from pyproj import Proj, transform
# conda install -c conda-forge proj4
from shapely.geometry import Point
from shapely import wkb
from shapely.wkt import loads
# https://pypi.org/project/geopy/
from geopy.geocoders import Nominatim
import folium
from folium import plugins

from dataplay.merge import mergeDatasets 

import dexplot as dxp

# In case file is KML, enable support
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

#this cell is good to copy
from shapely.geometry import LineString
pd.plotting.register_matplotlib_converters()

from dataplay.geoms import readInGeometryData
from shapely import wkt
from dataplay.geoms import readInGeometryData

import ipywidgets as widgets
!jupyter nbextension enable --py widgetsnbextension
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
alt.data_transformers.enable('default', max_rows=None)

</details>
<details>
<summary>

## Convenience Functions 

</summary>

In [ ]:
#hide 
# Return the boundaries of a Linestring 
def split(geom):
  print( str( type(geom)) )
  #Linestring might not be the actual type, so this may need to be fied. 
  #IF its not a linestring then dont run it and stuff 'false' and the datatype
  if str( type(geom)) == "<class 'shapely.geometry.linestring.LineString'>" and not str(geom.boundary) == 'MULTIPOINT EMPTY':
    print(geom.boundary)
    left, right = geom.boundary
    print('left x: ', type(left.x), left.x)
    print('left y: ', type(left.y), left.y)
    print('right x: ', type(right.x), right.x)
    print('right y: ', type(right.y), right.y)
    return left.x, left.y, right.x, right.y
  else: return False, type(geom), False, type(geom)

In [ ]:
# To 'import' a script you wrote, map its filepath into the sys
def getPointsInPolygons(pts, polygons, ptsCoordCol, polygonsCoordCol): 
    count = 0 
    total = pts.size / len(pts.columns)
    # We're going to keep a list of how many points we find.
    pts_in_polys = []

    # Loop over polygons with index i.
    for i, poly in polygons.iterrows():
        # print('Searching for point within Geom:', poly )
        # Keep a list of points in this poly
        pts_in_this_poly = []

        # Now loop over all points with index j.
        for j, pt in pts.iterrows():
          if poly[polygonsCoordCol].contains(pt[ptsCoordCol]):
            # Then it's a hit! Add it to the list,
            # and drop it so we have less hunting.
            pts_in_this_poly.append(pt[ptsCoordCol])
            pts = pts.drop([j])

        # We could do all sorts, like grab a property of the
        # points, but let's just append the number of them.
        pts_in_polys.append(len(pts_in_this_poly))
        print('Found this many points within the Geom:', len(pts_in_this_poly) ) 
        count = count + len(pts_in_this_poly) 
        clear_output(wait=True)

    # Add the number of points for each poly to the dataframe.
    polygons['pointsinpolygon'] = gpd.GeoSeries(pts_in_polys)
    print( 'Total Points: ', total )
    print( 'Total Points in Polygons: ', count )
    print( 'Prcnt Points in Polygons: ', count / total )
    return polygons

</details>
<details>
<summary>

## File Access Convenience Functions 

</summary>

In [ ]:
# Find Relative Path to Files
def findFile(root, file):
    for d, subD, f in os.walk(root):
        if file in f:
            return "{1}/{0}".format(file, d)
            break 

# To 'import' a script you wrote, map its filepath into the sys
def addPath(root, file): sys.path.append(os.path.abspath( findFile( './', file) ))

In [ ]:
findFile('./', 'Routing September 2019.geojson')

In [ ]:
ls

</details>
<details>
<summary>

## Inspect Deployment Data

</summary>

In [ ]:
ls 'Trip origins-destinations by month'

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.
fileName = "Trip Origins by block September 2019.geojson"  #@param ['Daily Deployment average by block August 2019.csv', 'Daily Deployment average by block December 2019.csv', 'Daily Deployment average by block November 2019.csv', 'Daily Deployment average by block October 2019.csv', 'Daily Deployment average by block September 2019.csv', 'Trip Destinations by block August 2019.geojson','Trip Destinations by block December 2019.geojson','Trip Destinations by block November 2019.geojson', 'Trip Destinations by block October 2019.geojson', 'Trip Destinations by block September 2019.geojson', 'Trip Origins by block August 2019.geojson','Trip Origins by block December 2019.geojson','Trip Origins by block November 2019.geojson','Trip Origins by block October 2019.geojson','Trip Origins by block September 2019.geojson']
#@markdown ---

In [ ]:
gdf = gpd.read_file( findFile('./', fileName) )
gdf.head()

In [ ]:
gdf.plot(column = 'value')

In [ ]:
point_df = gdf.copy()
point_df['centroids'] = df.centroid
point_df = point_df.drop(columns = 'geometry')
point_df = point_df.set_geometry('centroids')
point_df.head(1)
point_df.plot(marker='o', color='red', markersize='value')

In [ ]:
point_df.value.hist()

In [ ]:
point_df[point_df.value > 1000].plot()

In [ ]:
baltimore = gpd.read_file("https://opendata.arcgis.com/datasets/b738a8587b6d479a8824d937892701d8_0.geojson")

In [ ]:
from dataplay.geoms import workWithGeometryData

In [ ]:
baltimore.columns

In [ ]:
pointsInPolys = workWithGeometryData(method = 'ponp', df = point_df, polys = baltimore, ptsCoordCol ='centroids', 
                                     polygonsCoordCol = 'geometry', polygonsLabel = 'CSA2010')
pointsInPolys.plot(column='value', legend=True, markersize = 1)

In [ ]:
ponp_copy = pointsInPolys.copy()

In [ ]:
ponp_copy.value.isnull().groupby([ponp_copy['CSA2010']]).sum().astype(int).reset_index(name='NumberMissingCount').head()

In [ ]:
ponp_copy.value.notnull().groupby([ponp_copy['CSA2010']]).sum().astype(int) / ponp_copy.value.isnull().groupby([ponp_copy['CSA2010']]).sum().astype(int)

In [ ]:
ponp_copy.value.isnull().groupby([ponp_copy['CSA2010']]).sum().astype(int) / ponp_copy.value.groupby([ponp_copy['CSA2010']]).sum().astype(int) * 100

In [ ]:
ponp_copy.fillna(-1).groupby('CSA2010')['value'].sum()

In [ ]:
gdf.value.unique()

In [ ]:
gdf.value.describe()

In [ ]:
gdf.value.var() # Return unbiased variance over requested axis.

In [ ]:
gdf.value.sem() # Return unbiased standard error of the mean over requested axis.

In [ ]:
gdf.value.nunique()	# Count distinct observations over requested axis.

In [ ]:
# DataFrame.shape	Return a tuple representing the dimensionality of the DataFrame.
gdf.shape

#DataFrame.size	Return an int representing the number of elements in this object.
gdf.size

# DataFrame.ndim	Return an integer representing the number of axes/array dimensions.
gdf.ndim

# Note Used : 
# DataFrame.axes	Return a list representing the axes of the DataFrame.

gdf.dtypes

# Return unbiased kurtosis over requested axis using Fisher’s definition of kurtosis (kurtosis of normal == 0.0).
gdf.kurtosis()

</details>
<details>
<summary>

## Load Tracts

</summary>

In [ ]:
# This will just beautify the output.

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('max_colwidth', 20)

In [ ]:
# The attributes are what we will use.
in_crs = 2248 # The CRS we recieve our data.
out_crs = 4326 # The CRS we would like to have our data represented as.
geom = 'geometry' # The column where our spatial information lives.

# A Url to load
boundariesBaltimoreTractsNoWater2010 = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ8xXdUaT17jkdK0MWTJpg3GOy6jMWeaXTlguXNjCSb8Vr_FanSZQRaTU-m811fQz4kyMFK5wcahMNY/pub?gid=886223646&single=true&output=csv"

# Read in the dataframe
gdf = pd.read_csv(boundariesBaltimoreTractsNoWater2010)

# Convert the geometry column datatype from a string of text into a coordinate datatype.
gdf['geometry'] = gdf['geometry'].apply(lambda x: loads( str(x) ))

# Process the dataframe as a geodataframe with a known CRS and geom column.
gdf = GeoDataFrame(gdf, crs=in_crs, geometry='geometry')

In [ ]:
boundariesBaltimoreTractsNoWater2010.head()

Ensure merge is on consistent datatypes

In [ ]:
gdf['GEOID10'] = gdf['GEOID10'].astype(str)
scooterdf['nameChange2'] = scooterdf['nameChange2'].astype(str)

Perform the merge

In [ ]:
scooterdfClean = gdf.merge(scooterdf, left_on='GEOID10', right_on='nameChange2').drop(['name', 'nameChange1', 'nameChange2'], axis=1)

In [ ]:
scooterdfClean.head()

In [ ]:
# scooterdf.to_csv('./scooterdf.csv', index=False)
# gdf.drop(columns='geometry').to_csv('./boundsdf.csv', index=False)

In [ ]:
ls

In [ ]:
scooterdfClean.groupby('CSA')['value'].sum()

In [ ]:
scooterdfClean.value.isna().groupby([scooterdfClean['CSA']]).sum().astype(int).reset_index(name='notApplicable')

In [ ]:
scooterdfClean.value.notnull().groupby([scooterdfClean['CSA']]).sum().astype(int).reset_index(name='NotMissingCount')

In [ ]:
scooterdfClean.value.isnull().groupby([scooterdfClean['CSA']]).sum().astype(int).reset_index(name='NumberMissingCount')

In [ ]:
scooterdfClean.fillna(-1).groupby('CSA')['value'].sum()

In [ ]:
scooterdfClean.groupby('CSA')['value'].mean()

In [ ]:
scooterdfClean.groupby('CSA')['value'].sum()

In [ ]:
scooterdfClean.CSA.value_counts()

In [ ]:
https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html

</details>
<details>
<summary>

## Inspect Routes Data

</summary>

In [ ]:
fileName = 'Routing December 2019.geojson'  #@param ['Routing August 2019.geojson', 'Routing October 2019.geojson', 'Routing December 2019.geojson', 'Routing September 2019.geojson', 'Routing November 2019.geojson']
columnName = "streetname"  #@param ['id', 'color', 'streetname', 'trip_count_sum', 'trip_count_average', 'trip_count_percent']
gdf = gpd.read_file( findFile('./', fileName) )
gdf.head()

In [ ]:
baltimore = gpd.read_file("https://opendata.arcgis.com/datasets/b738a8587b6d479a8824d937892701d8_0.geojson")

In [ ]:
tst = """background = alt.Chart(gdf).mark_geoshape(
    stroke='white',
    strokeWidth=2
).encode(
    color=alt.value('#eee'),
).properties(
    width=700,
    height=500
)"""

# GeoDataFrame could be passed as usual pd.DataFrame
city = alt.Chart(baltimore).mark_geoshape(
).project(
).encode(
    color='tpop10', # shorthand infer types as for regular pd.DataFrame
    tooltip='CSA2010' # GeoDataFrame.index is accessible as id
).properties(
    width=500,
    height=300
)

routes = alt.Chart(gdf).mark_geoshape(
    filled=False,
    strokeWidth=2
)

city + routes

Clean the gdf of empties.

In [ ]:
gdf = gdf[~gdf.isna()]
gdf = gdf[~gdf.is_empty]

Now get the extremities; this will take a minute.

In [ ]:
# hide_output
gdf['lefty'], gdf['leftx'],gdf['righty'], gdf['rightx'] = zip(*gdf["geometry"].map(split))

Split the gdf into a left and right dataset

In [ ]:
gdf_left = gdf.copy()
gdf_left = gdf_left.drop(columns = ['geometry','rightx', 'righty'])

gdf_right= gdf.copy()
gdf_right = gdf_right.drop(columns = ['geometry','leftx', 'lefty', 'streetname', 'trip_count_sum', 'trip_count_average', 'trip_count_percent', 'color' ])

The coordinate variables of object type will cause problems.

In [ ]:
gdf_right.dtypes

Let's go ahead and coerce a correction.

In [ ]:
gdf_right['righty']=pd.to_numeric(gdf_right['righty'], errors='coerce')
gdf_left['lefty']=pd.to_numeric(gdf_left['lefty'], errors='coerce')

gdf_right['rightx']=pd.to_numeric(gdf_right['rightx'], errors='coerce')
gdf_left['leftx']=pd.to_numeric(gdf_left['leftx'], errors='coerce')

Now we can view the results

In [ ]:
gdf_right.dtypes

Save these csv's because it took a minute to get to where we are now.

In [ ]:
gdf_right.to_csv('rightRouts.csv')
gdf_left.to_csv('leftRouts.csv')

Convert the datasets to geodataframes for further exploration!

In [ ]:
# We could create a geodataframe like this
#temp_gdf = gpd.GeoDataFrame( gdf_right, geometry=gpd.points_from_xy(gdf_right.rightx, gdf_right.righty) )
#temp_gdf.head()

# Alternately this could work.. unfinished.. but wkt.loads can make a Point from text
# gdf_right['strCol']=gdf_right['rightx'].astype(str)
# gdf_right['geometry'] = gdf_right['strCol'].apply(wkt.loads)

In [ ]:
# hide_output
left_csaMap = readInGeometryData(url=gdf_left, porg='p', geom= False, lat= 'leftx', lng= 'lefty', revgeocode=False, save=False, in_crs=4268, out_crs=4268)

In [ ]:
# hide_output
right_csaMap = readInGeometryData(url=gdf_right, porg='p', geom= False, lat= 'rightx', lng= 'righty', revgeocode=False, save=False, in_crs=4268, out_crs=4268)

In [ ]:
right_csaMap.head()

In [ ]:
left_csaMap.head()

In [ ]:
left_csaMap.plot(column='trip_count_sum')

In [ ]:
right_csaMap.plot()

In [ ]:
baltimore.columns

In [ ]:
# hide_output
right_points_in_poly = getPointsInPolygons(right_csaMap, baltimore, 'geometry', 'geometry')

In [ ]:
right_csaMap.head()

In [ ]:
left_points_in_poly = getPointsInPolygons(left_csaMap, baltimore, 'geometry', 'geometry')

In [ ]:
left_csaMap.head()

In [ ]:
left_points_in_poly.drop('geometry', axis=1)[['pointsinpolygon']].head(20)

In [ ]:
right_points_in_poly.drop('geometry', axis=1)[['pointsinpolygon']].head(20)

In [ ]:
left_points_in_poly.plot(column='pointsinpolygon', legend=True)

In [ ]:
right_points_in_poly.plot(column='pointsinpolygon', legend=True)

In [ ]:
right_points_in_poly

In [ ]:
scooterdfClean = left_points_in_poly.merge(right_points_in_poly, left_on='CSA2010', right_on='CSA2010', how='left')

In [ ]:
scooterdfClean.columns

In [ ]:
right_points_in_poly.columns

In [ ]:
right_points_in_poly.head()

In [ ]:
right_points_in_poly.plot('pointsinpolygon', legend=True)

In [ ]:
import altair as alt
import geopandas as gpd
import gpdvega
# GeoDataFrame could be passed as usual pd.DataFrame
chart = alt.Chart(right_points_in_poly).mark_geoshape(
).project(
).encode(
    color='pointsinpolygon', # shorthand infer types as for regular pd.DataFrame
    tooltip=['CSA2010','pointsinpolygon'] # GeoDataFrame.index is accessible as id
).properties(
    width=500,
    height=300
) 

routes = alt.Chart(gdf).mark_geoshape(
    filled=False,
    strokeWidth=2
)

chart + routes

chart.save(fileName[:-8]+'.html', embed_options={'renderer':'svg'})
chart.save(fileName[:-8]+'.json')

</details>
<details>
<summary>

## Inspect Origins-Destinations Data

</summary>

In [ ]:
ls 'Trip origins-destinations by month'

In [ ]:
# @title Example form fields
#@markdown Forms support many types of fields.
fileName = 'Trip Destinations by block August 2019.geojson'  #@param ['Trip Destinations by block August 2019.geojson', 'Trip Destinations by block December 2019.geojson', 'Trip Destinations by block November 2019.geojson', 'Trip Destinations by block October 2019.geojson', 'Trip Destinations by block September 2019.geojson', 'Trip Origins by block August 2019.geojson', 'Trip Origins by block December 2019.geojson', 'Trip Origins by block November 2019.geojson', 'Trip Origins by block October 2019.geojson', 'Trip Origins by block September 2019.geojson']
columnName = "name"  #@param ['name', 'value', 'color', 'radius']

#@markdown ---

gdf = gpd.read_file( findFile('./', fileName) )

gdf.plot( column = columnName)
gdf.columns
gdf[['id','name', 'value', 'color', 'radius']].head(5)

In [ ]:
dxp.bar(x='color', y='value', data=gdf, aggfunc='median')

In [ ]:
dxp.scatter(x = "color", y = "value", data = gdf, aggfunc = "median")

</details>